In [2]:
import pandas as pd

# Given description

https://github.com/EgorDudyrev/SimMod_CourseWork/blob/master/Specification(%D0%B7%D0%B0%D0%B4%D0%B0%D1%87%D0%B0%20-%D0%91%D1%8E%D1%80%D0%BE%20%D0%BF%D1%83%D1%82%D0%B5%D1%88%D0%B5%D1%81%D1%82%D0%B2%D0%B8%D0%B9).docx

## Главная идея

Объединение нескольких бюро в 2 центра, принимающих звонки. Необходимо оптимизировать затраты на офисы: работу персонала и т.п.

Время работы:
* Office1 - 07:00-19:00
* Office2 - 19:00-07:00

Клиенты:
* Обычные
* Премиум
    * золотые
    * серебрянные
    
У премиумов - особый номер телефона. По этому номеру они и определяются

Количество линий связи - ограничено

Если линия занята, а клиент - обычный, то "нам очень важен ваш звонок". Система ведёт учёт таких случаев.

Если система загружена (используется большинство линий), оставшиеся линии - только vip'ам.

Сколько линий выделять под vip'ов - вот в чём вопрос

Время ожидания обычного клиента ~ треугольное распределение (min=12 минут, max= 30 минут)

Если оценочное время > того, что может терпеть клиент - он сбрасывается

Время ожидания vip ~ norm(min= 8 минут, max = 17 минут)

Клиенты сбросят вызов, если будут ждать слишком долго

Если звонит обычный клиент и линий нет:
* 8 секунд определяется время ожидания в очереди (клиент может сброситься в это время)

Если звонит vip и линия есть:
* 7-16 секунд - вводится номер карточки

Звонки есть 3х видов:
* запрос информации о путёвке
* бронь путёвки
* перебронирование

In [38]:
calls_type_distr = pd.DataFrame([['Вопрос', 0.16], ['Бронь', 0.76], ['Перебронь', 0.08]],
                                columns=['type', 'p'])

calls_type_distr

,type,p
0,Вопрос,0.16
1,Бронь,0.76
2,Перебронь,0.08


Операторы 3х типов:
* обычные
* серебрянные
* золотые - доступны только золотым клиентам

Клиенты 3х типов:
* обычные
* серебрянные - приоритет №2 - принимаются серебрянными и обычными - середина очереди
* золотые - приоритет №1 - принимаются всеми - в начале очереди

У операторов есть квалификации:
* золотой может уменьшить время обслуживания на 12%
* серебрянный может уменьшить время обслуживания на 5%

* Время на обслуживание - треугольное распределение
* Время на доп. работу - норм распределение

In [41]:
print('Время обслуживания клиентов (в минутах)')
service_stat_ds = pd.DataFrame()
service_stat_ds['type'] = ['Инфо', 'Бронь', 'Перебронь']
service_stat_ds['serv_time_regular'] = [(1.2, 2.05, 3.75),
                               (2.25, 2.95, 8.6),
                               (1.2, 1.9, 5.8)]
service_stat_ds['extra_time'] = [(0.05, 0.1),
                                 (0.5, 0.8),
                                 (0.4, 0.6)]
service_stat_ds

Время обслуживания клиентов


,type,serv_time_regular,extra_time
0,Инфо,"(1.2, 2.05, 3.75)","(0.05, 0.1)"
1,Бронь,"(2.25, 2.95, 8.6)","(0.5, 0.8)"
2,Перебронь,"(1.2, 1.9, 5.8)","(0.4, 0.6)"


In [32]:
calls_stat_ds = pd.DataFrame()
calls_stat_ds['time_range'] = range(7,19)
calls_stat_ds['regular_clients'] = [87, 165, 236, 323, 277, 440, 269, 342, 175, 273, 115,  56]
calls_stat_ds['vip_clients'] = [89, 243, 221, 180, 301, 490, 394, 347, 240, 269, 145,  69]
calls_stat_ds['silver_clients'] = 0.68*calls_stat_ds['vip_clients']
calls_stat_ds['gold_clients'] = calls_stat_ds['vip_clients']-calls_stat_ds['silver_clients']

In [34]:
print('Статистика по звонкам')
calls_stat_ds

Статистика по звонкам


,time_range,regular_clients,vip_clients,silver_clients,gold_clients
0,7,87,89,60.52,28.48
1,8,165,243,165.24,77.76
2,9,236,221,150.28,70.72
3,10,323,180,122.40,57.60
4,11,277,301,204.68,96.32
5,12,440,490,333.20,156.80
6,13,269,394,267.92,126.08
7,14,342,347,235.96,111.04
8,15,175,240,163.20,76.80
9,16,273,269,182.92,86.08
